### 框架干的最厉害的一件事就是帮我们把返向传播全部计算好了

In [1]:
import torch

需要求导的，可以手动定义：

In [2]:
#方法1
x = torch.randn(3,4,requires_grad=True)
x

tensor([[-1.1372, -0.5008,  0.7780, -0.8478],
        [ 1.3141, -0.0251,  0.7600, -0.2223],
        [-0.2428,  0.2850, -0.3272, -0.4244]], requires_grad=True)

In [3]:
#方法2
x = torch.randn(3,4)
x.requires_grad=True
x

tensor([[-1.0031,  1.8381,  0.4780, -1.0301],
        [ 0.2920, -0.6560, -1.4152, -0.9292],
        [ 1.3096, -0.1140,  1.1998,  0.4837]], requires_grad=True)

In [4]:
b = torch.randn(3,4,requires_grad=True)

In [5]:
t = x + b

In [6]:
y = t.sum()
y

tensor(2.7124, grad_fn=<SumBackward0>)

In [7]:
y.backward()

In [8]:
b.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

虽然没有指定t的requires_grad但是需要用到它，也会默认的

In [9]:
x.requires_grad, b.requires_grad, t.requires_grad

(True, True, True)

In [10]:
y.requires_grad

True

### 举个例子看一下：

![title](./img/2.png)

In [14]:
#计算流程
x = torch.rand(1)
b = torch.rand(1, requires_grad = True)
w = torch.rand(1, requires_grad = True)
y = w * x 
z = y + b
print(x)

tensor([0.8471])


In [15]:
x.requires_grad, b.requires_grad, w.requires_grad, y.requires_grad#注意y也是需要的

(False, True, True, True)

In [16]:
x.is_leaf, w.is_leaf, b.is_leaf, y.is_leaf, z.is_leaf

(True, True, True, False, False)

返向传播计算

In [23]:
z.backward(retain_graph=True)#如果不清空会累加起来

In [24]:
w.grad

tensor([2.5413])

In [25]:
b.grad

tensor([3.])

### 做一个线性回归试试水

构造一组输入数据X和其对应的标签y

In [26]:
import numpy as np

In [28]:
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
print(x_train)
x_train = x_train.reshape(-1, 1)#重整形状
x_train.shape

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


(11, 1)

In [29]:
y_values = [2*i + 1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

(11, 1)

In [30]:
import torch
import torch.nn as nn

### 线性回归模型
- 其实线性回归就是一个不加激活函数的全连接层

In [32]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

In [33]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)

In [34]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

指定好参数和损失函数

In [35]:
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

训练模型

In [36]:
for epoch in range(epochs):
    epoch += 1
    # 注意转行成tensor
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)

    # 梯度要清零每一次迭代
    optimizer.zero_grad() 

    # 前向传播
    outputs = model(inputs)

    # 计算损失
    loss = criterion(outputs, labels)

    # 返向传播
    loss.backward()

    # 更新权重参数
    optimizer.step()
    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 50, loss 0.014008061029016972
epoch 100, loss 0.007989726960659027
epoch 150, loss 0.00455700047314167
epoch 200, loss 0.002599137369543314
epoch 250, loss 0.0014824484242126346
epoch 300, loss 0.0008455259376205504
epoch 350, loss 0.0004822661285288632
epoch 400, loss 0.0002750591083895415
epoch 450, loss 0.00015688483836129308
epoch 500, loss 8.94818440428935e-05
epoch 550, loss 5.103841976961121e-05
epoch 600, loss 2.9107754016877152e-05
epoch 650, loss 1.6605283235548995e-05
epoch 700, loss 9.470030818192754e-06
epoch 750, loss 5.4013898989069276e-06
epoch 800, loss 3.081627255596686e-06
epoch 850, loss 1.7570547470313613e-06
epoch 900, loss 1.0023292134064832e-06
epoch 950, loss 5.714789494959405e-07
epoch 1000, loss 3.260503262936254e-07


### 测试模型预测结果

In [44]:
predicted = model(torch.from_numpy(x_train).requires_grad_())#.data.numpy()
predicted

tensor([[ 1.0011],
        [ 3.0009],
        [ 5.0008],
        [ 7.0006],
        [ 9.0005],
        [11.0003],
        [13.0001],
        [15.0000],
        [16.9998],
        [18.9997],
        [20.9995]], grad_fn=<AddmmBackward0>)

### 模型的保存与读取

In [38]:
torch.save(model.state_dict(), 'model.pkl')

In [39]:
model.load_state_dict(torch.load('model.pkl'))

<All keys matched successfully>

### 使用GPU进行训练
- 只需要把数据和模型传入到cuda里面就可以了

In [27]:
import torch
import torch.nn as nn
import numpy as np


class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim, output_dim)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


criterion = nn.MSELoss()


learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 1000
for epoch in range(epochs):
    epoch += 1
    inputs = torch.from_numpy(x_train).to(device)
    labels = torch.from_numpy(y_train).to(device)

    optimizer.zero_grad() 

    outputs = model(inputs)

    loss = criterion(outputs, labels)

    loss.backward()

    optimizer.step()

    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 50, loss 0.011100251227617264
epoch 100, loss 0.006331132724881172
epoch 150, loss 0.003611058695241809
epoch 200, loss 0.0020596047397702932
epoch 250, loss 0.0011747264070436358
epoch 300, loss 0.0006700288504362106
epoch 350, loss 0.00038215285167098045
epoch 400, loss 0.00021796672081109136
epoch 450, loss 0.00012431896175257862
epoch 500, loss 7.090995495673269e-05
epoch 550, loss 4.044298475491814e-05
epoch 600, loss 2.3066799258231185e-05
epoch 650, loss 1.3156819477444515e-05
epoch 700, loss 7.503344477299834e-06
epoch 750, loss 4.279831500753062e-06
epoch 800, loss 2.4414177914877655e-06
epoch 850, loss 1.3924694712841301e-06
epoch 900, loss 7.945647553242452e-07
epoch 950, loss 4.530382398115762e-07
epoch 1000, loss 2.5830334493548435e-07
